In [22]:
import streamlit as st
import numpy as np
import pandas as pd
import math

from PIL import Image, ImageDraw
from lego_colors import LEGO_COLORS_ALL
from lego_colors_round import LEGO_COLORS_ROUND
from lego_colors_square import LEGO_COLORS_SQUARE
from lego_colors_square_available import LEGO_COLORS_SQUARE_AVAILABLE
from utils import create_mosaic, draw_mosaic, draw_instructions, get_image_download_link, save_feedback_to_google_sheets


In [47]:
import math

def find_closest_lego_color(r, g, b, lego_colors):
    """Find the closest LEGO color to the given RGB input using CIE Lab color space."""
    # Clamp RGB values to [0, 255]
    r, g, b = max(0, min(255, float(r))), max(0, min(255, float(g))), max(0, min(255, float(b)))
    
    # Convert input RGB to Lab
    input_lab = rgb_to_lab(r, g, b)
    
    # Compute closest color by CIE Lab color space distance
    min_dist = float('inf')
    closest = None
    for name, hex_code, lr, lg, lb in lego_colors:
        color_lab = rgb_to_lab(lr, lg, lb)
        # Calculate Delta E (CIE76 color difference)
        dist = math.sqrt(
            (input_lab[0] - color_lab[0]) ** 2 + 
            (input_lab[1] - color_lab[1]) ** 2 + 
            (input_lab[2] - color_lab[2]) ** 2
        )
        if dist < min_dist:
            min_dist = dist
            closest = {
                "name": name,
                "hex": hex_code,
                "rgb": (lr, lg, lb),
                "distance": dist
            }
    return closest

def rgb_to_xyz(r, g, b):
    """Convert RGB to XYZ color space."""
    # Normalize RGB values
    r = r / 255
    g = g / 255
    b = b / 255
    
    # Apply gamma correction
    r = r ** 2.2 if r > 0.04045 else r / 12.92
    g = g ** 2.2 if g > 0.04045 else g / 12.92
    b = b ** 2.2 if b > 0.04045 else b / 12.92
    
    # Convert to XYZ using sRGB matrices
    r *= 100
    g *= 100
    b *= 100
    
    x = r * 0.4124 + g * 0.3576 + b * 0.1805
    y = r * 0.2126 + g * 0.7152 + b * 0.0722
    z = r * 0.0193 + g * 0.1192 + b * 0.9505
    
    return [x, y, z]

def xyz_to_lab(x, y, z):
    """Convert XYZ to Lab color space."""
    # Use D65 white point
    xn = 95.047
    yn = 100.0
    zn = 108.883
    
    # Normalize XYZ values
    x = x / xn
    y = y / yn
    z = z / zn
    
    # Apply cube root approximation
    x = x ** (1/3) if x > 0.008856 else (7.787 * x) + (16/116)
    y = y ** (1/3) if y > 0.008856 else (7.787 * y) + (16/116)
    z = z ** (1/3) if z > 0.008856 else (7.787 * z) + (16/116)
    
    L = (116 * y) - 16
    a = 500 * (x - y)
    b = 200 * (y - z)
    
    return [L, a, b]

def rgb_to_lab(r, g, b):
    """Convert RGB directly to Lab color space."""
    xyz = rgb_to_xyz(r, g, b)
    return xyz_to_lab(xyz[0], xyz[1], xyz[2])

In [48]:

find_closest_lego_color(228, 228, 228, LEGO_COLORS_SQUARE_AVAILABLE)



{'name': 'Lavender',
 'hex': '#E1D5ED',
 'rgb': (225, 213, 237),
 'distance': 13.506084021504547}